In [11]:
import ccxt
import os
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
# import sys
# import shutil
# from functools import partial
# from pathlib import Path
import sys
import shutil
from collections import Counter


exchange = ccxt.binance()

exchange.load_markets()

symbols = exchange.symbols

In [12]:
t = 'USDT'
pairs = []
# global unwanted
unwanted = ["XNO","BTT","GYEN","UP/","DOWN/","USDS","USDT/","USDSB","SUSD",
"NCASH/ETH","DNT/ETH","BCN/ETH","UST","ETH/","BKRW","VAI","RUB",
"NGN","DAI","BIDR","BEAR", "BULL", "PAX", "TUSD", "UMA", "USDC",
 "USDP", "BUSD/", "EUR", "GBP", "TRY", "AUD","BRL", "BVND"]

for s in symbols:
    if (t in s):
        pairs.append(s)

pairs=[n for n in pairs if all(m not in n for m in unwanted)]


_pairs = ["ATOM/USDT"]


In [13]:

# always put the list out of the loop or everthing will be wiped out each turn

timeframes = ["4h", "8h", "1d", "1h"]
# timeframes = ["8h"]
# limit = 1003

# td =datetime.now().strftime('%Y-%m-%d_%H')
mydir=f"{os.getcwd()}/data"
if os.path.exists(mydir):
    shutil.rmtree(mydir)

for timeframe in timeframes:

    for symbol in tqdm(pairs):

        m_symbol = symbol.replace("/","")
        outname = f"{m_symbol}_{timeframe}.csv"

        # outdir=f"{os.getcwd()}/data/{td}/{timeframe}"
        outdir=f"{os.getcwd()}/data/{timeframe}"
        # outdir=f"{os.getenv('HOME')}/data/{td}/{timeframe}"
        fullname = os.path.join(outdir, outname) 

        # remove old dir
        # if os.path.exists(outdir):
        #     shutil.rmtree(outdir)

        if not os.path.exists(outdir):
            os.makedirs(outdir)

        bars = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=999)
        # must use bars[:-1] because arrgrelextrema will see the last candle wich have not closed yet
        data = pd.DataFrame(bars[:-1], columns=['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
        data['Time'] = pd.to_datetime(data['Time'], unit='ms')
        data.set_index('Time', inplace=True)
        # if len(data) < 22:
        #     print("data less than 22 candles for ", symbol)
        #     continue
        if len(data) > 22:
            data.to_csv(fullname) 
        # print('fetching new data', symbol)
    
# run the script 11 minutes before the znd of the current hour or 1 m after the hour
# or you will face problems with datetime




100%|██████████| 317/317 [02:23<00:00,  2.20it/s]


In [14]:
# remove dfs that stoppped updating 

for timeframe in timeframes:
    outdir2 = f"{os.getcwd()}/data/{timeframe}"
    values=[]
    keys = []
    result = {}
    dicts = {}
    
    for filename in os.listdir(outdir2):
        symbol2 = filename.split("_")[0]
        df = pd.read_csv(f"{outdir2}/{filename}", index_col='Time')
        values.append(df.index[-1])
        keys.append(symbol2)

    dicts=dict(zip(keys, values))


    res = Counter(dicts.values()).most_common(1)
    res2 = res[0][0]

    # # the problem here
    for key,value in dicts.items():
        if res2 != value:
            # result[key] = value
            # os.remove(fullname)

            # show the coins removed
            # print(value, key)

            # print(f"{outdir2}/{key}_{timeframe}.csv")
            os.remove(f"{outdir2}/{key}_{timeframe}.csv")




In [15]:
print(data.index[-1])

2022-02-21 12:00:00
